In [65]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import LabelEncoder

# Lade dein Dataset
data = pd.read_csv('Housing.csv')


# Kodieren der kategorischen Merkmale in numerische Werte
label_encoder = LabelEncoder()
categorical_cols = ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea', 'furnishingstatus']

for col in categorical_cols:
    data[col] = label_encoder.fit_transform(data[col])

# Splitte das Dataset in Features (X) und Target (y)
X = data.drop('price', axis=1)  # Features (alle Spalten außer 'price')
y = data['price']  # Zielvariable (Preis)

# Funktion zum Bootstrappen und Testen verschiedener Regressionsmodelle
def bootstrap_regression(X, y, models, n_iterations=100):
    predictions = {model_name: [] for model_name in models.keys()}
    
    for i in range(n_iterations):
        # Generiere ein Bootstrap-Sample (mit Ersetzung)
        sample_X = X.sample(n=len(X), replace=True)
        sample_y = y.loc[sample_X.index]
        
        for model_name, model in models.items():
            # Trainiere das Modell auf dem Bootstrap-Sample
            model.fit(sample_X, sample_y)
            
            # Mache Vorhersagen auf dem gesamten Datensatz oder einem Test-Set
            predictions[model_name].append(model.predict(X))
    
    # Berechne den Durchschnitt der Vorhersagen für jedes Modell
    mean_predictions = {model_name: np.mean(np.array(pred_list), axis=0) for model_name, pred_list in predictions.items()}
    
    return mean_predictions

# Definiere die Regressionsmodelle, die du testen möchtest
models = {
    'Linear Regression': LinearRegression(),
    'Decision Tree': DecisionTreeRegressor(max_depth=5,random_state=42),
    'Random Forest': RandomForestRegressor(n_estimators=100,max_depth=5,random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, max_depth=5,random_state=42)
}

# Führe das Bootstrapping aus und teste die Modelle
mean_predictions = bootstrap_regression(X, y, models, n_iterations=100)

# Bewerte jedes Modell und berechne die MSE und R^2
for model_name, predictions in mean_predictions.items():
    mse = mean_squared_error(y, predictions)
    mae = mean_absolute_error(y, predictions)
    r_squared = 1 - mse / np.var(y)
    
    print(f"{model_name} - Mean Absolute Error: {mae}")
    print(f"{model_name} - Mean Squared Error: {mse}")
    print(f"{model_name} - R^2: {r_squared}\n")


Linear Regression - Mean Absolute Error: 777048.1628619065
Linear Regression - Mean Squared Error: 1117817343961.227
Linear Regression - R^2: 0.679903399426071

Decision Tree - Mean Absolute Error: 655012.705113263
Decision Tree - Mean Squared Error: 764641303320.7579
Decision Tree - R^2: 0.781038393102726

Random Forest - Mean Absolute Error: 665687.2158928746
Random Forest - Mean Squared Error: 803749670497.2633
Random Forest - R^2: 0.7698393761481004

Gradient Boosting - Mean Absolute Error: 408342.2673994449
Gradient Boosting - Mean Squared Error: 297966598351.79987
Gradient Boosting - R^2: 0.9146747044745294



In [63]:
# Lade den ersten Eintrag aus der CSV-Datei (angenommen, die Datei heißt 'Housing.csv')
new_entry = pd.read_csv('Housing.csv').iloc[100]
new_entry

price                   6230000
area                       6600
bedrooms                      3
bathrooms                     2
stories                       1
mainroad                    yes
guestroom                    no
basement                    yes
hotwaterheating              no
airconditioning             yes
parking                       0
prefarea                    yes
furnishingstatus    unfurnished
Name: 100, dtype: object

In [64]:

new_entry = new_entry.drop('price')  # Entferne die 'price' Spalte, da sie nicht benötigt wird

# Kodieren der kategorischen Merkmale in numerische Werte
label_encoder = LabelEncoder()
categorical_cols = ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea', 'furnishingstatus']

# Wenden des LabelEncoders auf jede kategorische Spalte an
for col in categorical_cols:
    # Sicherstellen, dass wir die Spalte als Serie übergeben und sie in 'new_entry' korrekt umwandeln
    new_entry[col] = label_encoder.fit_transform([new_entry[col]])

# Überprüfen des vorverarbeiteten neuen Eintrags
print(new_entry)

# Vorhersage für den neuen Eintrag mit allen getesteten Modellen
for model_name, model in models.items():
    # Die Eingabedaten müssen als 2D-Array übergeben werden (Matrix-Form)
    prediction = model.predict([new_entry])
    print(f"{model_name} - Vorhergesagter Preis: {prediction[0]}")


area                6600
bedrooms               3
bathrooms              2
stories                1
mainroad               0
guestroom              0
basement               0
hotwaterheating        0
airconditioning        0
parking                0
prefarea               0
furnishingstatus       0
Name: 100, dtype: object
Linear Regression - Vorhergesagter Preis: 4712933.208337621
Decision Tree - Vorhergesagter Preis: 6206565.217391305
Random Forest - Vorhergesagter Preis: 6090022.579342162
Gradient Boosting - Vorhergesagter Preis: 5091014.69228576


/opt/anaconda3/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(
